# Generic CNN for MNIST
The goal of this notebook ist to generalize the necessary building blocks for a CNN for our MNIST. This allows fast experimentation with different network designs. We also seek to integrate Tensorboard support for this. 

  Ideally it allows us to phase out all complexity related to the general purpose apis so that we only need to focus on the abstract data that actually differs between different network designs.

## Dense to one hot
https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/learn/python/learn/datasets/mnist.py

In [1]:
def dense_to_one_hot(labels_dense, num_classes):
  """Convert class labels from scalars to one-hot vectors."""
  num_labels = labels_dense.shape[0]
  index_offset = numpy.arange(num_labels) * num_classes
  labels_one_hot = numpy.zeros((num_labels, num_classes))
  labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
  return labels_one_hot


## First Approach
We base our first approach on the tutorial for TensorBoard with MNIST.

The original code comes from https://raw.githubusercontent.com/tensorflow/tensorflow/r0.11/tensorflow/examples/tutorials/mnist/mnist_with_summaries.py.

We use it as a base to iterate on.

In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
from functools import *
from datetime import datetime

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

class dotdict(dict):
    __getattr__ = dict.get


def logdir(d,name=""): 
    return "/".join([d, name , datetime.now().strftime("%Y%m%d-%H%M%S")])
    
FLAGS = dotdict({'fake_data':False,
                 'max_steps':1000,
                 'learning_rate':0.001,
                 'dropout':0.9,
                 'data_dir':'/tmp/data',
                 'summaries_dir':'/tmp/mnist_logs',
                 'epochs' : 1 })

n_inputs = 784
n_classes = 10
batch_size = 50

#setup placeholders
def placeholder():    
    x = tf.placeholder('float',[None,n_inputs])
    y = tf.placeholder('float',[None,n_classes])
    return (x,y)

x,y=placeholder()

def variable_summaries(var, name):
    """Attach a lot of summaries to a Tensor."""
    with tf.name_scope('summaries'):
        with tf.name_scope(name):
            mean = tf.reduce_mean(var)
            tf.scalar_summary('mean/' + name, mean)
            with tf.name_scope('stddev'):
                stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
            tf.scalar_summary('stddev/' + name, stddev)
            tf.scalar_summary('max/' + name, tf.reduce_max(var))
            tf.scalar_summary('min/' + name, tf.reduce_min(var))
            tf.histogram_summary(name, var)

def weight(w,name,**kwargs):
    with tf.name_scope('weights'):
        """Create a weight variable with appropriate initialization."""
        initial = tf.truncated_normal(w, stddev=0.1)
        ret =  tf.Variable(initial)
        variable_summaries(ret,name+'/weights')
        return ret
    
def bias(b,name,**kwargs):
    """Create a bias variable with appropriate initialization."""
    with tf.name_scope('bias'):
        initial = tf.constant(0.1, shape=b)
        ret = tf.Variable(initial)
        variable_summaries(ret,name+'/bias')
        return ret
        
def maxpool2d(x,name, k_w=2,k_s=2,**kwargs):
    with tf.name_scope('max_pool'):
        ret = tf.nn.max_pool(x, ksize=[1, k_w, k_w, 1], strides=[1, k_s, k_s, 1],
                          padding='SAME')
        tf.histogram_summary(name+'/max_pool',ret)
        return ret

def conv2d(x, w, b, k=1):
    x = tf.nn.conv2d(x, w, strides=[1, k, k, 1] , padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def conv(x,w,b,name,k_c=1,**kwargs):
    # Convolution
    return conv2d(x,w, b,k=k_c)

def fc(x,w,b,name,k_c=1,k_p=2,**kwargs):
    with tf.name_scope('Wx_plus_b'):
        pre= tf.add(tf.matmul(x, w), b)
        tf.histogram_summary(name+'/pre_activations',pre)
        return pre

def relu(pre,name,**kwargs):
    with tf.name_scope('activations'):
        ret = tf.nn.relu(pre)
        tf.histogram_summary(name+'/activations',ret)
        return ret
    
def log_conv(conv,name,**kwargs):
    _,_,img_size,channels = conv.get_shape().as_list()
    ## Prepare for visualization
    # Take only convolutions of first image, discard convolutions for other images.
    V = tf.slice(conv, (0, 0, 0, 0), (1, -1, -1, -1), name='slice_first_input')
    V = tf.reshape(V, (img_size, img_size, channels))
    # Reorder so the channels are in the first dimension, x and y follow.
    V = tf.transpose(V, (2, 0, 1))
    # Bring into shape expected by image_summary
    V = tf.reshape(V, (-1, img_size, img_size, 1))
    tf.image_summary(name+'/convolved',V,10)
    return conv

layer_type = ({'conv':conv, 'fc':fc}).get
post_type = ({'maxpool2d':maxpool2d,'relu':relu, 'log_conv':log_conv}).get

def conv_neural_net(x,lshp , lt=layer_type):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    def process_layer(**args):
        wl,bl=(i(**args) for i in (weight,bias))
        acc = lt(args['fn'])(**{**args,'w':wl,'b':bl})
        if 'post' in args:
            acc = reduce(lambda a,e: post_type(e[0])(a,args['name'],*e[1:]),args['post'],acc)
        return acc
    def fun(acc,e):
        prev, cur= e
        with tf.name_scope(cur['name']):
            if 'fn' in prev:
                if (prev['fn'] == 'conv' and cur['fn']!='conv'):
                    acc = tf.reshape(acc,[-1,cur['w'][0]])
                if (prev['fn']!='conv' and cur['fn']=='conv'):
                    acc = tf.reshape(acc, shape=[-1, 28, 28, 1])
            return process_layer(**{**cur, 'x':acc})
    mixed = zip([{},*lshp],lshp)
    return reduce(fun,mixed,x)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
#list of stacked layers
#name is name for logging, w is weight shape, b is bias shape,
# fn is name for layer type (or rather layer construction function)
layer_std = [
    {'name':'c1','w':[5,5,1,32],'b':[32],'fn':'conv','post':[('relu',),('maxpool2d',),('log_conv',)]},
    {'name':'c2', 'w':[5,5,32,64], 'b':[64],'fn':'conv','post':[('relu',),('maxpool2d',)]},
    {'name':'d1','w':[7*7*64,1024], 'b':[1024],'fn':'fc','post':[('relu',)]},
    {'name':'out','w':[1024,n_classes], 'b':[n_classes],'fn':'fc'},
]

In [3]:
#list of stacked layers
#name is name for logging, w is weight shape, b is bias shape,
# fn is name for layer type (or rather layer construction function)
layer_deep = [
    {'name':'c1','w':[5,5,1,32],'b':[32],'fn':'conv','post':[('relu',),('maxpool2d',),('log_conv',)]},
    {'name':'c2','w':[3,3,32,64],'b':[64],'fn':'conv','post':[('relu',),('maxpool2d',)]},
    {'name':'c3','w':[3,3,64,64],'b':[64],'fn':'conv','post':[('relu',)]},
    {'name':'c4','w':[3,3,64,64],'b':[64],'fn':'conv','post':[('relu',)]},
    {'name':'d1','w':[7*7*64,1024], 'b':[1024],'fn':'fc','post':[('relu',)]},
    {'name':'out','w':[1024,n_classes], 'b':[n_classes],'fn':'fc'},
]

In [4]:
layer_deep_2 = [
    {'name':'c1','w':[7,7,1,32],'b':[32],'fn':'conv','post':[('relu',),('maxpool2d',),('log_conv',)]},
    {'name':'c2','w':[5,5,32,64],'b':[64],'fn':'conv','post':[('relu',),('maxpool2d',)]},
    {'name':'c3','w':[3,3,64,64],'b':[64],'fn':'conv','post':[('relu',)]},
    {'name':'c4','w':[3,3,64,64],'b':[64],'fn':'conv','post':[('relu',)]},
    {'name':'c5','w':[3,3,64,64],'b':[64],'fn':'conv','post':[('relu',)]},
    {'name':'d1','w':[7*7*64,1024], 'b':[1024],'fn':'fc','post':[('relu',)]},
    {'name':'out','w':[1024,n_classes], 'b':[n_classes],'fn':'fc'},
]

In [5]:
min_deep = [
    {'name':'c1','w':[3,3,1,16],'b':[16],'fn':'conv','post':[('relu',),('maxpool2d',),('log_conv',)]},
    {'name':'c2','w':[3,3,16,16],'b':[16],'fn':'conv','post':[('relu',),('maxpool2d',)]},
    {'name':'c4','w':[3,3,16,16],'b':[16],'fn':'conv','post':[('relu',)]},
    {'name':'c5','w':[3,3,16,8],'b':[8],'fn':'conv','post':[('relu',),('log_conv',)]},
    {'name':'d1','w':[7*7*8,128], 'b':[128],'fn':'fc','post':[('relu',)]},
    {'name':'out','w':[128,n_classes], 'b':[n_classes],'fn':'fc'},
]

In [6]:
ultra_deep = [
    {'name':'c1','w':[3,3,1,16],'b':[16],'fn':'conv','post':[('relu',),('log_conv',)]},
    {'name':'c2','w':[3,3,16,16],'b':[16],'fn':'conv','post':[('relu',)]},
    {'name':'c3','w':[3,3,16,16],'b':[16],'fn':'conv','post':[('relu',)]},
    {'name':'c4','w':[3,3,16,16],'b':[16],'fn':'conv','post':[('relu',)]},
    {'name':'c5','w':[3,3,16,16],'b':[16],'fn':'conv','post':[('relu',)]},
    {'name':'c6','w':[3,3,16,16],'b':[16],'fn':'conv','post':[('relu',)]},
    {'name':'c7','w':[3,3,16,16],'b':[16],'fn':'conv','post':[('relu',),('maxpool2d',)]},
    {'name':'c8','w':[3,3,16,8],'b':[8],'fn':'conv','post':[('relu',),('maxpool2d',),('log_conv',)]},
    {'name':'d1','w':[7*7*8,128], 'b':[128],'fn':'fc','post':[('relu',)]},
    {'name':'out','w':[128,n_classes], 'b':[n_classes],'fn':'fc'},
]

In [7]:
ultra_deep_2 = [
    {'name':'c1','w':[3,3,1,64],'b':[64],'fn':'conv','post':[('relu',),('maxpool2d',),('log_conv',)]},
    {'name':'c2','w':[3,3,64,128],'b':[128],'fn':'conv','post':[('relu',),('maxpool2d',)]},
    {'name':'c3','w':[3,3,128,16],'b':[16],'fn':'conv','post':[('relu',)]},
    {'name':'c4','w':[3,3,16,16],'b':[16],'fn':'conv','post':[('relu',)]},
    {'name':'c5','w':[3,3,16,16],'b':[16],'fn':'conv','post':[('relu',)]},
    {'name':'c6','w':[3,3,16,16],'b':[16],'fn':'conv','post':[('relu',)]},
    {'name':'c7','w':[3,3,16,16],'b':[16],'fn':'conv','post':[('relu',)]},
    {'name':'c8','w':[3,3,16,8],'b':[8],'fn':'conv','post':[('relu',),('log_conv',)]},
    {'name':'d1','w':[7*7*8,128], 'b':[128],'fn':'fc','post':[('relu',)]},
    {'name':'out','w':[128,n_classes], 'b':[n_classes],'fn':'fc'},
]

In [8]:
layers = {
    'std':layer_std,
    'deep_1':layer_deep,
    'deep_2':layer_deep_2,
    'min_deep':min_deep
}

In [9]:
def cost(pred,y):
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred,y))
        tf.scalar_summary('cross entropy',loss)
        return loss
    
def opt(loss,learning_rate):
    with tf.name_scope('optimizer'):
        return tf.train.AdamOptimizer(learning_rate).minimize(loss)

def wrong_imgs(pred,y,x):
    with tf.name_scope('correct'):
        x_ = tf.reshape(x, shape=[-1, 28, 28, 1])
        false = tf.not_equal(tf.argmax(pred,1), tf.argmax(y,1))
        filtered = tf.boolean_mask(x_,false)
        return tf.image_summary('misclassified',filtered,10)
        
        
def acc(pred,y):
    with tf.name_scope('accuracy'):
        correct = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        tf.scalar_summary('accuracy',accuracy)
        return accuracy

In [10]:
def train_and_test_conv_neural_net(x, y,pred, summaries_dir,learning_rate,epochs = 1,fake_data=True,**kwargs):
    loss = cost(pred,y)
    optim = opt(loss,learning_rate)
    accuracy = acc(pred,y)
    merged = tf.merge_all_summaries()
    wrong_images =wrong_imgs(pred,y,x)
    with tf.Session() as sess:
        train_writer = tf.train.SummaryWriter(summaries_dir + '/train',sess.graph)
        test_writer = tf.train.SummaryWriter(summaries_dir + '/test')
        sess.run(tf.initialize_all_variables())
        idx = lambda e,i: int(mnist.train.num_examples/batch_size*e+i)
        for e in range(epochs):
            epoch_loss = 0
            for i in range(int(mnist.train.num_examples/batch_size)):
                if i % 1000 == 999:
                    print("actually got here in 1 epoch")
                    summary, accu, w_i = sess.run([merged,accuracy,wrong_images],feed_dict = {x:mnist.test.images, y:mnist.test.labels})
                    [test_writer.add_summary(s,idx(e,i)) for s in (summary, w_i)]
                if i%20 == 19:
                    epoch_x, epoch_y = mnist.train.next_batch(batch_size,fake_data)
                    _, c,summary = sess.run([optim, loss,merged], feed_dict = {x:epoch_x, y:epoch_y})
                    train_writer.add_summary(summary, idx(e,i))
                    epoch_loss = epoch_loss + c
                else:
                    epoch_x, epoch_y = mnist.train.next_batch(batch_size, fake_data)
                    _, c = sess.run([optim, loss], feed_dict = {x:epoch_x, y:epoch_y})
                    epoch_loss = epoch_loss + c
            print('Epoch ' + str(e+1) + ' out of ' + str(epochs) + ' / Loss: ' + str(epoch_loss))
            summary, accu, w_i = sess.run([merged,accuracy,wrong_images],feed_dict = {x:mnist.test.images, y:mnist.test.labels})
            [test_writer.add_summary(s,idx(e+1,0)) for s in (summary, w_i)]
            print('Accuracy: ' + str(accu))
        print('Training finished!')

In [27]:
def train_layer(name,layer_shape):
    tf.reset_default_graph()
    x,y = placeholder()
    pred = conv_neural_net(x,layer_shape)
    summaries_dir = logdir(FLAGS['summaries_dir'],name)
    print("Writing to: "+summaries_dir)
    train_and_test_conv_neural_net(x,y,pred,**{**FLAGS, 'epochs': 5,'summaries_dir':summaries_dir})
    
def train_layers(layer_shapes):
    return [train_layer(n,l) for (n,l) in layer_shapes.items()]

In [15]:
train_layer("std",layer_std)

Writing to: /tmp/mnist_logs/std/20161206-151259
actually got here in 1 epoch
Epoch 1 out of 3 / Loss: 249.640399276
Accuracy: 0.9827
actually got here in 1 epoch
Epoch 2 out of 3 / Loss: 51.5672024529
Accuracy: 0.9831
actually got here in 1 epoch
Epoch 3 out of 3 / Loss: 33.8092931461
Accuracy: 0.9881
Training finished!


In [ ]:
train_layer("ultra_deep",ultra_deep)

Writing to: /tmp/mnist_logs/ultra_deep/20161206-144904


In [32]:
train_layer("min_deep", min_deep)

Writing to: /tmp/mnist_logs/min_deep/20161206-144007
Epoch 1 out of 3 / Loss: 318.539597624
Accuracy: 0.9572
Epoch 2 out of 3 / Loss: 88.32974363
Accuracy: 0.9821
Epoch 3 out of 3 / Loss: 59.1003624431
Accuracy: 0.9833
Training finished!


In [12]:
min_deep_2 = [
    {'name':'c1','w':[3,3,1,16],'b':[16],'fn':'conv','post':[('relu',),('maxpool2d',),('log_conv',)]},
    {'name':'c2','w':[3,3,16,16],'b':[16],'fn':'conv','post':[('relu',),('maxpool2d',)]},
    {'name':'c5','w':[3,3,16,8],'b':[8],'fn':'conv','post':[('relu',),('log_conv',)]},
    {'name':'d1','w':[7*7*8,128], 'b':[128],'fn':'fc','post':[('relu',)]},
    {'name':'out','w':[128,n_classes], 'b':[n_classes],'fn':'fc'},
]

In [17]:
train_layer("min_deep_2",min_deep_2)

Writing to: /tmp/mnist_logs/min_deep_2/20161206-155933
actually got here in 1 epoch
Epoch 1 out of 3 / Loss: 307.399545964
Accuracy: 0.9665
actually got here in 1 epoch
Epoch 2 out of 3 / Loss: 81.6447817045
Accuracy: 0.9836
actually got here in 1 epoch
Epoch 3 out of 3 / Loss: 57.4519388558
Accuracy: 0.9851
Training finished!


In [13]:
min_deep_3 = [
    {'name':'c1','w':[3,3,1,16],'b':[16],'fn':'conv','post':[('relu',),('maxpool2d',),('log_conv',)]},
    {'name':'c2','w':[3,3,16,16],'b':[16],'fn':'conv','post':[('relu',),('maxpool2d',)]},
    {'name':'c5','w':[3,3,16,8],'b':[8],'fn':'conv','post':[('relu',),('maxpool2d',),('log_conv',)]},
    {'name':'d1','w':[4*4*8,64], 'b':[64],'fn':'fc','post':[('relu',)]},
    {'name':'out','w':[64,n_classes], 'b':[n_classes],'fn':'fc'},
]

In [27]:
train_layer("min_deep_3",min_deep_3)

Writing to: /tmp/mnist_logs/min_deep_3/20161206-160620
actually got here in 1 epoch
Epoch 1 out of 3 / Loss: 415.470117016
Accuracy: 0.9669
actually got here in 1 epoch
Epoch 2 out of 3 / Loss: 114.922457585
Accuracy: 0.9781
actually got here in 1 epoch
Epoch 3 out of 3 / Loss: 84.8723762429
Accuracy: 0.9766
Training finished!


In [28]:
min_deep_4 = [
    {'name':'c1','w':[3,3,1,4],'b':[4],'fn':'conv','post':[('relu',),('maxpool2d',),('log_conv',)]},
    {'name':'c2','w':[3,3,4,8],'b':[8],'fn':'conv','post':[('relu',),('maxpool2d',)]},
    {'name':'c5','w':[3,3,8,4],'b':[4],'fn':'conv','post':[('relu',),('maxpool2d',),('log_conv',)]},
    {'name':'d1','w':[4*4*4,8], 'b':[8],'fn':'fc','post':[('relu',)]},
    {'name':'out','w':[8,n_classes], 'b':[n_classes],'fn':'fc'},
]

In [29]:
train_layer("min_deep_4",min_deep_4)

Writing to: /tmp/mnist_logs/min_deep_4/20161206-172628
actually got here in 1 epoch
Epoch 1 out of 5 / Loss: 836.503729656
Accuracy: 0.922
actually got here in 1 epoch
Epoch 2 out of 5 / Loss: 256.473767843
Accuracy: 0.9429
actually got here in 1 epoch
Epoch 3 out of 5 / Loss: 203.766857659
Accuracy: 0.9537
actually got here in 1 epoch
Epoch 4 out of 5 / Loss: 180.892082803
Accuracy: 0.9547
actually got here in 1 epoch
Epoch 5 out of 5 / Loss: 162.45784852
Accuracy: 0.9601
Training finished!


In [31]:
min_deep_5 = [
    {'name':'c1','w':[3,3,1,4],'b':[4],'fn':'conv','post':[('relu',),('maxpool2d',),('log_conv',)]},
    {'name':'c2','w':[3,3,8,4],'b':[4],'fn':'conv','post':[('relu',),('maxpool2d',4,4),('log_conv',)]},
    {'name':'d1','w':[4*4*4,8], 'b':[8],'fn':'fc','post':[('relu',)]},
    {'name':'out','w':[8,n_classes], 'b':[n_classes],'fn':'fc'},
]

In [32]:
train_layer("min_deep_5",min_deep_5)

ValueError: Dimensions must be equal, but are 4 and 8

In [9]:
train_layers(layers)

Writing to: /tmp/mnist_logs/layer_std20161202-211729
Epoch 1 out of 3 / Loss: 243.785580625
Accuracy: 0.9702
Epoch 2 out of 3 / Loss: 47.4891263249
Accuracy: 0.9856
Epoch 3 out of 3 / Loss: 31.0756102614
Accuracy: 0.9874
Training finished!
Writing to: /tmp/mnist_logs/layers_deep20161202-214140
Epoch 1 out of 3 / Loss: 220.648349631
Accuracy: 0.9839
Epoch 2 out of 3 / Loss: 57.6456857131
Accuracy: 0.9848
Epoch 3 out of 3 / Loss: 43.3919358567
Accuracy: 0.9855
Training finished!


[None, None]

In [14]:
[(k,v) for (k,v) in layers.items()]

[('layer_std',
  [{'b': [32],
    'fn': 'conv',
    'name': 'c1',
    'post': [('relu',), ('maxpool2d',)],
    'w': [5, 5, 1, 32]},
   {'b': [64],
    'fn': 'conv',
    'name': 'c2',
    'post': [('relu',), ('maxpool2d',)],
    'w': [5, 5, 32, 64]},
   {'b': [1024],
    'fn': 'fc',
    'name': 'd1',
    'post': [('relu',)],
    'w': [3136, 1024]},
   {'b': [10], 'fn': 'fc', 'name': 'out', 'w': [1024, 10]}]),
 ('layers_deep',
  [{'b': [32],
    'fn': 'conv',
    'name': 'c1',
    'post': [('relu',), ('maxpool2d',)],
    'w': [5, 5, 1, 32]},
   {'b': [64],
    'fn': 'conv',
    'name': 'c2',
    'post': [('relu',), ('maxpool2d',)],
    'w': [3, 3, 32, 64]},
   {'b': [64],
    'fn': 'conv',
    'name': 'c3',
    'post': [('relu',), ('maxpool2d',)],
    'w': [3, 3, 64, 64]},
   {'b': [64],
    'fn': 'conv',
    'name': 'c4',
    'post': [('relu',), ('maxpool2d',)],
    'w': [3, 3, 64, 64]},
   {'b': [1024],
    'fn': 'fc',
    'name': 'd1',
    'post': [('relu',)],
    'w': [3136, 1024]},


In [6]:
train_and_test_conv_neural_net(x,y,pred,**{**FLAGS, 'epochs': 3})

Epoch 1 out of 3 / Loss: 241.791304823
Accuracy: 0.9641
Epoch 2 out of 3 / Loss: 50.770959126
Accuracy: 0.9886
Epoch 3 out of 3 / Loss: 32.847308922
Accuracy: 0.9884
Training finished!


## Misc Legacy Snippets
please ignore

In [1]:
# modules:
def inputs():
    pass
    
def inference():
    pass
    
def loss():
    pass

def train():
    pass

def weight_variable(shape):
    """Create a weight variable with appropriate initialization."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def variable_summaries(var, name):
    """Attach a lot of summaries to a Tensor."""
    with tf.name_scope('summaries'):
      mean = tf.reduce_mean(var)
      tf.scalar_summary('mean/' + name, mean)
      with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
      tf.scalar_summary('stddev/' + name, stddev)
      tf.scalar_summary('max/' + name, tf.reduce_max(var))
      tf.scalar_summary('min/' + name, tf.reduce_min(var))
      tf.histogram_summary(name, var)

def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    """Reusable code for making a simple neural net layer.

    It does a matrix multiply, bias add, and then uses relu to nonlinearize.
    It also sets up name scoping so that the resultant graph is easy to read,
    and adds a number of summary ops.
    """
    # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.name_scope(layer_name):
      # This Variable will hold the state of the weights for the layer
      with tf.name_scope('weights'):
        weights = weight_variable([input_dim, output_dim])
        variable_summaries(weights, layer_name + '/weights')
      with tf.name_scope('biases'):
        biases = bias_variable([output_dim])
        variable_summaries(biases, layer_name + '/biases')
      with tf.name_scope('Wx_plus_b'):
        preactivate = tf.matmul(input_tensor, weights) + biases
        tf.histogram_summary(layer_name + '/pre_activations', preactivate)
      activations = act(preactivate, name='activation')
      tf.histogram_summary(layer_name + '/activations', activations)
      return activations

def train():
  # Import data
  mnist = input_data.read_data_sets(FLAGS.data_dir,
                                    one_hot=True,
                                    fake_data=FLAGS.fake_data)

  sess = tf.InteractiveSession()

  # Create a multilayer model.

  # Input placeholders
  with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, 10], name='y-input')

  with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.image_summary('input', image_shaped_input, 10)

  

  hidden1 = nn_layer(x, 784, 500, 'layer1')

  with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    tf.scalar_summary('dropout_keep_probability', keep_prob)
    dropped = tf.nn.dropout(hidden1, keep_prob)

  # Do not apply softmax activation yet, see below.
  y = nn_layer(dropped, 500, 10, 'layer2', act=tf.identity)

  with tf.name_scope('cross_entropy'):
    # The raw formulation of cross-entropy,
    #
    # tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.softmax(y)),
    #                               reduction_indices=[1]))
    #
    # can be numerically unstable.
    #
    # So here we use tf.nn.softmax_cross_entropy_with_logits on the
    # raw outputs of the nn_layer above, and then average across
    # the batch.
    diff = tf.nn.softmax_cross_entropy_with_logits(y, y_)
    with tf.name_scope('total'):
      cross_entropy = tf.reduce_mean(diff)
    tf.scalar_summary('cross entropy', cross_entropy)

  with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(FLAGS.learning_rate).minimize(
        cross_entropy)

  with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
      correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    with tf.name_scope('accuracy'):
      accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.scalar_summary('accuracy', accuracy)

  # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
  merged = tf.merge_all_summaries()
  train_writer = tf.train.SummaryWriter(FLAGS.summaries_dir + '/train',
                                        sess.graph)
  test_writer = tf.train.SummaryWriter(FLAGS.summaries_dir + '/test')
  tf.initialize_all_variables().run()

  # Train the model, and also write summaries.
  # Every 10th step, measure test-set accuracy, and write test summaries
  # All other steps, run train_step on training data, & add training summaries

  def feed_dict(train):
    """Make a TensorFlow feed_dict: maps data onto Tensor placeholders."""
    if train or FLAGS.fake_data:
      xs, ys = mnist.train.next_batch(100, fake_data=FLAGS.fake_data)
      k = FLAGS.dropout
    else:
      xs, ys = mnist.test.images, mnist.test.labels
      k = 1.0
    return {x: xs, y_: ys, keep_prob: k}

  for i in range(FLAGS.max_steps):
    if i % 10 == 0:  # Record summaries and test-set accuracy
      summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict(False))
      test_writer.add_summary(summary, i)
      print('Accuracy at step %s: %s' % (i, acc))
    else:  # Record train set summaries, and train
      if i % 100 == 99:  # Record execution stats
        run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
        run_metadata = tf.RunMetadata()
        summary, _ = sess.run([merged, train_step],
                              feed_dict=feed_dict(True),
                              options=run_options,
                              run_metadata=run_metadata)
        train_writer.add_run_metadata(run_metadata, 'step%03d' % i)
        train_writer.add_summary(summary, i)
        print('Adding run metadata for', i)
      else:  # Record a summary
        summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True))
        train_writer.add_summary(summary, i)
  train_writer.close()
  test_writer.close()


def main():
  if tf.gfile.Exists(FLAGS.summaries_dir):
    tf.gfile.DeleteRecursively(FLAGS.summaries_dir)
  tf.gfile.MakeDirs(FLAGS.summaries_dir)
  train()

In [2]:
main()

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Accuracy at step 0: 0.0438
Accuracy at step 10: 0.6836
Accuracy at step 20: 0.8173
Accuracy at step 30: 0.8474
Accuracy at step 40: 0.8707
Accuracy at step 50: 0.8793
Accuracy at step 60: 0.8848
Accuracy at step 70: 0.8819
Accuracy at step 80: 0.884
Accuracy at step 90: 0.8966
Adding run metadata for 99
Accuracy at step 100: 0.9085
Accuracy at step 110: 0.9136
Accuracy at step 120: 0.9173
Accuracy at step 130: 0.9179
Accuracy at step 140: 0.9208
Accuracy at step 150: 0.9164
Accuracy at step 160: 0.9279
Accuracy at step 170: 0.9243
Accuracy at step 180: 0.9224
Accuracy at step 190: 0.9219
Adding run metadata for 199
Accuracy at step 200: 0.9274
Accuracy at step 210: 0.9304
Accuracy at step 220: 0.9347
Accuracy at step 230: 0.9353
Accuracy at step 240: 0.9352
Accuracy at step 250: 0.9307
Accuracy at s